In [ ]:
# Install required packages
!pip install -q ultralytics kagglehub

print("Packages installed successfully!")

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Navigate to project folder
work_dir = '/content/drive/MyDrive/Aircraft_Detect'
os.chdir(work_dir)

print(f"Working directory: {work_dir}")

# Check if dataset exists
dataset_path = 'dataset'
if os.path.exists(dataset_path):
    print(f"Dataset found")
    
    # Show dataset splits
    for split in ['train', 'valid', 'test']:
        split_path = os.path.join(dataset_path, split, 'images')
        if os.path.exists(split_path):
            count = len(os.listdir(split_path))
            print(f"  {split}: {count} images")
    
    # Create data.yaml if it doesn't exist
    if not os.path.exists('data.yaml'):
        print("\n⚠ data.yaml not found - creating it...")
        data_yaml = f"""path: {os.path.abspath(dataset_path)}
train: train/images
val: valid/images
test: test/images

names:
  0: aircraft
  1: drone
  2: helicopter

nc: 3
"""
        with open('data.yaml', 'w') as f:
            f.write(data_yaml)
        print(" data.yaml created!")
    else:
        print(" data.yaml found")
else:
    print("\n Dataset not found!")
    print("  Run pull.ipynb first to download and prepare the dataset.")

In [1]:
from ultralytics import YOLO

# Load pretrained YOLOv11 nano model (will auto-download if not present)
print("Loading YOLOv11n model...")
model = YOLO('yolo11n.pt')  # Updated to correct model name
print(" Model loaded!")

In [ ]:
# Train the model (with resume support)
import os
import glob

# Find the most recent checkpoint
checkpoint_patterns = [
    'runs/detect/train*/weights/last.pt',
    'exp*/*/weights/last.pt'
]

checkpoint_path = None
for pattern in checkpoint_patterns:
    checkpoints = glob.glob(pattern)
    if checkpoints:
        # Get the most recent checkpoint
        checkpoint_path = max(checkpoints, key=os.path.getmtime)
        break

if checkpoint_path and os.path.exists(checkpoint_path):
    print(f" Checkpoint found: {checkpoint_path}")
    print("Resuming training from last saved state...\n")
    
    # Load checkpoint and resume with cache enabled
    model = YOLO(checkpoint_path)
    results = model.train(resume=True, cache='disk')
    
else:
    print("No checkpoint found - starting fresh training...\n")
    
    # Start new training
    results = model.train(
        data='data.yaml',
        epochs=100,
        imgsz=640,
        batch=-1,
        patience=50,
        save=True,
        plots=True,
        cache='disk',
        verbose=True
    )

print("\n Training completed!")
print(f"Best model: {model.trainer.best}")
print(f"Last checkpoint: {model.trainer.last}")

In [ ]:
# Evaluate the model on validation set
metrics = model.val()

print("\nValidation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

In [ ]:
# Test inference on validation images
import os

val_images_dir = 'dataset/valid/images'
test_images = [os.path.join(val_images_dir, f) for f in os.listdir(val_images_dir)[:5]]

# Run inference
results = model.predict(test_images, save=True, conf=0.25)

print(f"\nInference completed on {len(test_images)} images")
print("Results saved to 'runs/detect/predict'")